In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import os.path

if not os.path.isfile('models.py'):
    from google.colab import drive
    drive.mount('/content/drive')

!pip install pandas_summary
#!pip install --upgrade pandas

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

## CARGA DE DATOS

In [ ]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model

In [ ]:
if not os.path.isfile('models.py'):
    df = pd.read_feather('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/train_normalized_data2.fth')
    df_test = pd.read_feather('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/test_normalized_data2.fth')
else:
    df = pd.read_feather('train_normalized_data2.fth')
    df_test = pd.read_feather('test_normalized_data2.fth')

### DEFINICION DE FEATURES Y OUTPUS

In [ ]:
#cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', #'Promo2Weeks', 
#            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
#            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'SchoolHoliday_fw', #'SchoolHoliday_bw']

cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

    

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [ ]:
cat= {'Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks',           'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw'}
type(cat)

set

In [ ]:
#contin_vars = ['CompetitionDistance', 
#   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
#   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
#   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
#   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

contin_vars = ['CompetitionDistance', 
'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']   
# contin_vars = []

In [ ]:
y_out_columns = ['Sales']

### SPLIT DE DATASET

In [ ]:
# split_type = 'random'
#split_type = 'no_split'
split_type = 'last_week'

In [ ]:
# Esta es para entrenar con todo
if split_type == 'no_split':
    df_train = df
elif split_type == 'last_week':
    # Esto divide en train y val, como venimos haciendo
    df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
    df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')
elif split_type == 'random':
    # Splitting aleatorio
    np.random.seed(42)
    indexes = np.arange(len(df))
    np.random.shuffle(indexes)
    N = len(df)//5
    df_train = df[N:]
    df_val = df[:N]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')


#df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
#df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
#print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [ ]:
X_train = df_train[cat_vars + contin_vars]
if split_type != 'no_split':
    X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

### PRE_PROCESAMIENTO DE SALIDA

In [ ]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    if split_type != 'no_split':
        y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    if split_type != 'no_split':
        y_val = (df_val[y_out_columns].values - y_mean)/y_std

## Categorical columns transformation (SOLO PARA XGBOOST)


In [ ]:
# categoricals_processing = 'no_categoricals'
# categoricals_processing = 'use_onehotencoding'
#categoricals_processing = 'use_embeddings'
categoricals_processing = 'NONE'

In [ ]:
# XGBoost no admite features categoricos, por eso hay que hacer un pre-procesamiento para poder incluirlos

# El uso de embeddings, en este caso utilizados los embeddings generado MLP de 06-full-model
if categoricals_processing == 'use_embeddings':
    if not os.path.isfile('models.py'):
      embeddings_model = load_model('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/embeddings_model2.hdf5') #Cargo los embeddin hasta realizar el concatenate
    else:
      embeddings_model = load_model('embeddings_model.hdf5') #Cargo los embeddin hasta realizar el concatenate
    X_train = embeddings_model.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1) #pasa de un matriz de (814150, 40) de DataSet a un array de 40 listas
    if split_type != 'no_split':
      X_val = embeddings_model.predict(np.hsplit(X_val, X_val.shape[1]), verbose=1)
    X_test = embeddings_model.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

In [ ]:
if categoricals_processing == 'use_onehotencoding':
    # Use One Hot Encoding
    categorical_feature_mask = [col in cat_vars for col in X_train.columns]
    ohe = OneHotEncoder(categorical_features = categorical_feature_mask)
    ohe.fit(X_train)
    X_train = ohe.transform(X_train)
    if split_type != 'no_split':
      X_val = ohe.transform(X_val)
    X_test = ohe.transform(X_test)

## MODELOS SIN HYPESEARCH

In [ ]:

from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

### LIGHTGBM

In [ ]:
min_child_samples=5
n_estimators=1000
learning_rate=0.05
model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate )
#PARAMETROS DEL MODELO (FIT)
fit_params={#"early_stopping_rounds":100, 
            #"eval_metric" : 'l2', 
            #"eval_set" : [(X_val, y_val.reshape(-1))],
            #'eval_names': ['valid'],            
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }
#model.fit(X_train, y_train.reshape(-1), **fit_params)

#score = np.mean(cross_val_score(model, X_train, y_train.reshape(-1), cv=4, n_jobs=1,
#                                    scoring='neg_root_mean_squared_error', fit_params= fit_params))


#predictions =  cross_validate(model, X_train, y_train.reshape(-1), cv=4, n_jobs=1, scoring='neg_root_mean_squared_error', fit_params= fit_params)


In [ ]:
def predicctionDesNormalizacion(model):
  if log_output:
      y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
      y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
      y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
  else:
      y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
      y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
      y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

  return y_pred_train, y_pred, y_pred_test

y_pred_train, y_pred, y_pred_test = predicctionDesNormalizacion(model)

#print('RMSE en train:',np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train)))
#print('RMSE en val:', np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred)))

NotFittedError: ignored

## HYPER-SEARCH

In [ ]:
!pip install scikit-optimize
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Integer, Categorical
from skopt.plots import plot_convergence
from skopt.utils import use_named_args
from skopt import dump, load
from skopt import BayesSearchCV
from skopt.callbacks import CheckpointSaver
from skopt import load


from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
import hyperopt

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer



### Hyperparameters search

*   [Guía de Hyper para LightGBM](https://neptune.ai/blog/lightgbm-parameters-guide)




### SPACES SEARCH

In [ ]:
### SKOPT-LIGHTGBM

dim_learning_rate= Real(low=0.01, high=1, prior='log-uniform', name='learning_rate')
dim_max_depth= Integer(0, 500, name='max_depth')
dim_num_leaves= Integer(2, 150,name='num_leaves') #max number of leaves in one tree        
dim_min_child_samples= Integer(0, 200, name='min_child_samples') # minimal number of data in one leaf
dim_reg_lambda= Real(1e-9, 1000, 'log-uniform', name='reg_lambda') # L2 regularization
dim_reg_alpha= Real(1e-9, 1.0, 'log-uniform', name='reg_alpha') # L1 regularization
dim_colsample_bytree= Real(0.01, 1.0, 'uniform', name='colsample_bytree') # enabler of bagging fraction
dim_min_child_weight=Integer(0, 10, name='min_child_weight') # minimal number of data in one leaf.
dim_n_estimators= Integer(10, 1000, name='n_estimators') # cant. de estimadores secuenciales (se pone alto stopea earlystopping)        
dimensions = [
    dim_learning_rate,
    dim_max_depth,
    dim_num_leaves,
    dim_min_child_samples,
    dim_reg_lambda,
    dim_reg_alpha,
    dim_colsample_bytree,
    dim_min_child_weight,
    dim_n_estimators
        #'max_bin': Integer(100, 100000), # max number of bins that feature values will be bucketed
        #'subsample': Real(0.01, 1.0, 'uniform'),
        #'subsample_freq': Integer(0, 10), # bagging fraction        
        #'subsample_for_bin': Integer(100000, 500000), # number of data that sampled for histogram bins        
        #'scale_pos_weight': Real(1e-6, 500, 'log-uniform'), 
        
]


### BayesSearchCV
search_spacesBayesCV = {
    'learning_rate': Real(low=0.01, high=1, prior='log-uniform', name='learning_rate'),
    'max_depth': Integer(0, 500, name='max_depth'),
    'num_leaves': Integer(2, 150,name='num_leaves'), #max number of leaves in one tree        
    'min_child_samples': Integer(0, 200, name='min_child_samples'), # minimal number of data in one leaf
    'reg_lambda': Real(1e-9, 1000, 'log-uniform', name='reg_lambda'), # L2 regularization
    'reg_alpha': Real(1e-9, 1.0, 'log-uniform', name='reg_alpha'), # L1 regularization
    'colsample_bytree': Real(0.01, 1.0, 'uniform', name='colsample_bytree'), # enabler of bagging fraction
    'min_child_weight':Integer(0, 10, name='min_child_weight'), # minimal number of data in one leaf.
    'n_estimators': Integer(10, 1000, name='n_estimators') # cant. de estimadores secuenciales (se pone alto stopea earlystopping)    
}



### HyperOptSearch
search_HyperOptSearch = {
    'learning_rate': hp.loguniform(low=0.01, high=1, label='learning_rate'),
    'max_depth':  hp.choice('max_depth', list(np.arange(0, 500))),
    'num_leaves': hp.choice('num_leaves', list(np.arange(2, 150))), #max number of leaves in one tree        
    'min_child_samples': hp.choice('min_child_samples', list(np.arange(0, 200))), # minimal number of data in one leaf
    'reg_lambda': hp.loguniform('reg_lambda',1e-9, 1000), # L2 regularization
    'reg_alpha': hp.loguniform('reg_alpha',1e-9, 1.0), # L1 regularization
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0), # enabler of bagging fraction
    'min_child_weight': hp.choice('min_child_weight', list(np.arange(0, 10,))), # minimal number of data in one leaf.
    'n_estimators': hp.choice('n_estimators', list(np.arange(10, 1000))) # cant. de estimadores secuenciales (se pone alto stopea earlystopping)    
}


max_iterSearch = 500

### Boosting Function

####  LGBMRegressor y Skopt.gp_minimize

In [ ]:

best_accuracy = -10.0
best_parameters = [{'teste': 1}]

# Funcion para LightGBM y skopt
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, max_depth,num_leaves, min_child_samples,reg_lambda, reg_alpha, colsample_bytree, min_child_weight, n_estimators):
            #,boosting_type, , subsample_for_bin,           
    
    #print("learning rate: {:.7f}".format(learning_rate))
    #print("max depth:", max_depth)    
    #print("num leaves:", num_leaves)    
    #print("min child samples:", min_child_samples)
    #print("reg alpha: {:.7f}".format(reg_alpha))
    #print("reg lambda: {:.7f}".format(reg_lambda))    
    #print("colsample bytree: {:.7f}".format(colsample_bytree))
    #print("min child weight:", min_child_weight)
    #print("n estimators:", n_estimators)
    #print()
    

    fit_params={#"early_stopping_rounds":100, 
              #"eval_metric" : 'l2', 
              #"eval_set" : [(X_val, y_val.reshape(-1))],
              #'eval_names': ['valid'],
              'verbose': 0,
              'feature_name': 'auto', # that's actually the default
              'categorical_feature': cat_vars
}
    model = LGBMRegressor(learning_rate=learning_rate, max_depth=max_depth, num_leaves=num_leaves, 
                          min_child_samples=min_child_samples,reg_lambda=reg_lambda, reg_alpha=reg_alpha,
                          colsample_bytree=colsample_bytree, min_child_weight=min_child_weight, n_estimators=n_estimators, n_jobs=8)
    
    #fitted_model = model.fit(X_train, y_train, **fit_params)


    #score = -cross_val_score(model, X_train, y_train.reshape(-1), fit_params= fit_params).mean()

    score = np.mean(cross_val_score(model, X_train, y_train.reshape(-1), cv=4, n_jobs=1,
                                    scoring='neg_root_mean_squared_error', fit_params= fit_params))
    
    #pred = model.predict_proba(X_test)[:, 1]
    #score = roc_auc_score(y_test, pred)
    
    #print()
    #print("RMSE score: {:.8f}".format(score))
    #print()
    
    global best_accuracy
    global best_parameters
    
    if score > best_accuracy:
        best_parameters[0] = model.get_params()
        
        best_accuracy = score
        
    del model
    
    return score

In [ ]:
#%time search_result = gp_minimize(func=fitness, dimensions=dimensions,n_calls=max_iterSearch)#max_iterSearch)


In [ ]:
best_accuracy

In [ ]:
from skopt.plots import plot_convergence

plot_convergence(search_result)


In [ ]:
best_parameters

In [ ]:
search_result.fun

In [ ]:
print("""Best parameters:

    - learning_rate=%.6f
    - max_depth=%d
    - num_leaves=%d
    - min_child_samples=%d
    - reg_lambda=%.6f
    - reg_alpha=%.6f
    - colsample_bytree=%.6f
    - min_child_weight=%.6f
    - n_estimators=%d""" % (search_result.x[0], search_result.x[1],
                            search_result.x[2], search_result.x[3],
                            search_result.x[4], search_result.x[5], search_result.x[6]
                            , search_result.x[7], search_result.x[8]))


In [ ]:
model = LGBMRegressor(learning_rate=0.06803043658780875, max_depth=163, num_leaves=60, 
                          min_child_samples=127,reg_lambda=2.0761902130891506e-07, reg_alpha=0.006861106252963912,
                          colsample_bytree=0.7374138536412973, min_child_weight=7 ,n_estimators=847, n_jobs=8)
model.fit(X_train, y_train.reshape(-1), **fit_params)
y_pred_train, y_pred, y_pred_test = predicctionDesNormalizacion(model)

print('RMSE en train:',np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train)))
print('RMSE en val:', np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred)))

In [ ]:
model = LGBMRegressor(learning_rate=0.010361, max_depth=119, num_leaves=5, 
                          min_child_samples=66,reg_lambda=0, reg_alpha=0.001115,
                          colsample_bytree=0.408144, min_child_weight=1, n_estimators=922,n_jobs=8)
model.fit(X_train, y_train.reshape(-1), **fit_params)
y_pred_train, y_pred, y_pred_test = predicctionDesNormalizacion(model)

print('RMSE en train:',np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train)))
print('RMSE en val:', np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred)))

####  LGBMRegressor y BayesSearchCV

In [ ]:
lightGBMSearchBayeCV = LGBMRegressor(n_jobs=8,verbose=1)
bayesSearchOptLightGBM = BayesSearchCV(lightGBMSearchBayeCV,
                    search_spacesBayesCV,
                    scoring='neg_root_mean_squared_error',
                    cv=4,
                    n_iter=max_iterSearch,
                    n_jobs=8,
                    return_train_score=False,
                    refit=True,
                    fit_params= fit_params,
                    random_state=22)
%time bayesCVLightGBMFit =  bayesSearchOptLightGBM.fit(X_train, y_train.reshape(-1))

In [ ]:
print('best parameters:' bayesSearchOptLightGBM.best_params_)
print('best score':, bayesSearchOptLightGBM.best_score_)

In [ ]:
model = LGBMRegressor(learning_rate=0.010717, max_depth=1, num_leaves=60, 
                          min_child_samples=163,reg_lambda=280.438324, reg_alpha=0.000558,
                          colsample_bytree=0.259107, min_child_weight=3, n_estimators=731,n_jobs=8)
model.fit(X_train, y_train.reshape(-1), **fit_params)
y_pred_train, y_pred, y_pred_test = predicctionDesNormalizacion(model)

print('RMSE en train:',np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train)))
print('RMSE en val:', np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred)))

####  LGBMRegressor y HyperOpt

In [ ]:

def objective(params):
        
    clf = LGBMRegressor(**params)    
    score = -cross_val_score(clf, X_train.values, y_train.reshape(-1), cv=4, scoring='neg_root_mean_squared_error', n_jobs=1, verbose=1).mean()
    return score

tpe_trials = Trials()    


%time hyperOptLightGBM = fmin(fn=objective, space=search_HyperOptSearch, verbose=1, algo=hyperopt.tpe.suggest, max_evals=max_iterSearch)

  0%|          | 0/500 [00:00<?, ?it/s, best loss: ?]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



In [ ]:
print('best parameters:', hyperOptLightGBM)

NameError: ignored

In [ ]:
model = LGBMRegressor(**hyperOptLightGBM)
model.fit(X_train, y_train.reshape(-1), **fit_params)
y_pred_train, y_pred, y_pred_test = predicctionDesNormalizacion(model)

print('RMSE en train:',np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train)))
print('RMSE en val:', np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred)))

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Assortment', 'CompetitionMonthsOpen', 'CompetitionOpenSinceYear', 'Day', 'DayOfWeek', 'Events', 'Month', 'Promo2SinceYear', 'Promo2Weeks', 'PromoInterval', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'SchoolHoliday_fw', 'State', 'StateHoliday', 'StateHoliday_bool_bw', 'StateHoliday_bool_fw', 'Store', 'StoreType', 'Week', 'Year']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


RMSE en train: 0.5609264880370579
RMSE en val: 0.48367124004717155


## XGBoost

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [ ]:
n_estimators=4000
learning_rate=0.25
model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, 
                     objective='reg:squarederror', n_jobs=12, max_depth=8, gamma=0.005)
model

In [ ]:
fit_params={#"early_stopping_rounds":100, 
            #"eval_metric" : 'rmse', 
            #"eval_set" : [(X_val, y_val.reshape(-1))],
            'verbose': 100,
           }

In [ ]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [ ]:
score = model.score(X_val, y_val)

In [ ]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train)*max_log_y)
    y_pred = np.exp(model.predict(X_val)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test)*max_log_y)
else:
    y_pred_train = model.predict(X_train)*y_std + y_mean
    y_pred = model.predict(X_val)*y_std + y_mean
    y_pred_test = model.predict(X_test)*y_std + y_mean

In [ ]:
# Train
train_RMSE = np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [ ]:
# Validación
val_RMSE = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [ ]:
print(score, train_RMSE, val_RMSE)

# HYPER-SEARCH

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [ ]:
fit_params={#"early_stopping_rounds":100, 
            #"eval_metric" : 'rmse', 
            #"eval_set" : [(X_val, y_val.reshape(-1))],
            'verbose': 100,
           }
#Parametros para realizar la busqueda en BayesSearchCV
param_bayes = {
        'max_depth': Integer(1,31),
        'learning_rate': Real(np.exp(-6), np.exp(-1), prior='log-uniform'),
        'reg_lambda': Real(np.exp(-8), np.exp(-3), prior='log-uniform'),
        'gamma': Real(np.exp(-8), np.exp(-3), prior='log-uniform'),
        'min_child_samples': Integer(1, 20),
        'min_child_weight': Real(np.exp(-8), np.exp(-1))  }
opt = BayesSearchCV(XGBRegressor(n_estimators=4000, n_jobs=8, objective='reg:squarederror'), param_bayes, n_iter=32,random_state=0, n_jobs=-1, verbose=1, fit_params=fit_params)


fitBayesSearchCV = opt.fit(X_train, y_train.reshape(-1))

In [ ]:
#summarize results
print("Best: %f using %s" % (fitBayesSearchCV.best_score_, fitBayesSearchCV.best_params_))
means = fitBayesSearchCV.cv_results_['mean_test_score']
stds = fitBayesSearchCV.cv_results_['std_test_score']
params = fitBayesSearchCV.cv_results_['params']
print('--------------------------------------------------------------')
for mean, stdev, param in zip(means, stds, params):    
    print("%f (%f) with: %r" % (mean, stdev, param))

# Baseline

In [ ]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [ ]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [ ]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [ ]:
df_test[['Store', 'Sales']].head(10)

In [ ]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

In [ ]:
sample_csv['Sales'] = df_test['Sales']

In [ ]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [ ]:
sample_csv.head()

# Sumbit a la competición

In [ ]:
sample_csv = pd.read_csv(r'./dataset/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_{log_output}-{n_estimators}-{learning_rate}.csv', index=False)

In [ ]:
#SOCRE 0.24541

In [ ]:
cat_vars